In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the fhuA gene
model0['lb_list']['EX_fecrm_e ']=0
model0['lb_list']['EX_fecrm_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe2+ intake
                                        if 'EX_fe2_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([16,  5, 15,  2, 14,  8, 18,  5, 11, 14, 15,  1,  1, 20, 17,  9,  7,
        8,  4, 13, 12, 11,  1, 10, 19,  5,  3, 15, 10,  4]), 1: array([ 3, 10, 12, 14,  5, 18,  2, 11,  5, 16, 19, 10,  2, 15,  8, 17, 17,
       17,  9,  6, 13, 16,  4,  4,  9, 18, 18,  1, 19,  6])}
1
strain_number:  0 11.366666666666667
strain_various:  0 6.828778970081125
strain_number:  1 10.8
strain_various:  1 5.8503561145170195
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([16,  5, 15,  2, 14,  8, 18,  5, 11, 14, 15,  1,  1, 20, 17,  9,  7,
        8,  4, 13, 12, 11,  1, 10, 19,  5,  3, 15, 10,  4]), 1: array([ 3, 10, 12, 14,  5, 18,  2, 11,  5, 16, 19, 10,  2, 15,  8, 17, 17,
       17,  9,  6, 13, 16,  4,  4,  9, 18, 18,  1, 19,  6])}
_2:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862

glc__D_e_1:  -0.3292282383040819
glc__D_e_1:  -9.068134479278005
glc__D_e_1:  -36.017658504141345
glc__D_e_1:  -0.033259038279442876
glc__D_e_1:  -14.327857325217783
glc__D_e_1:  -29.426631452641114
glc__D_e_1:  -16.00390545598491
glc__D_e_1:  -22.905320805212856
glc__D_e_1:  -38.28593822694198
glc__D_e_1:  -20.912701900723928
glc__D_e_1:  -0.200416399151365
glc__D_e_1:  -13.77996158124335
glc__D_e_1:  -23.5406751503888
glc__D_e_1:  -1.11131495282339
glc__D_e_1:  -14.617170599698666
glc__D_e_1:  -19.724448963962022
glc__D_e_1:  -1.8767229556834124
glc__D_e_1:  -9.575217945241913
glc__D_e_1:  -6.4678292433177855
glc__D_e_1:  -0.1691203580381373
glc__D_e_1:  -15.224962989004062
glc__D_e_1:  -4.165595111841448
glc__D_e_1:  -0.9213090194196676
glc__D_e_1:  -10.561367559672036
glc__D_e_1:  -5.626073082652422
glc__D_e_1:  -1.6569035259297848
glc__D_e_1:  -9.452865788716178
glc__D_e_1:  -4.665545486701973
glc__D_e_1:  -9.462333664799111
glc__D_e_1:  -3.2974803099648398
glc__D_e_1:  -0.0531942

glc__D_e_1:  -2.9498890014643067
glc__D_e_1:  -0.6224102807349425
glc__D_e_1:  -0.8721353189170371
glc__D_e_1:  -2.9487274877383376
glc__D_e_1:  -1.4524078532067755
glc__D_e_1:  -0.8924521835982864
glc__D_e_1:  -0.5146489559245353
glc__D_e_1:  -1.090312211926816
glc__D_e_1:  -1.0723731950194784
glc__D_e_1:  -1.0474173425809996
glc__D_e_1:  -2.4666234346326603
glc__D_e_1:  -2.8538279190920455
glc__D_e_1:  -1.264391652063439
glc__D_e_1:  -3.1386827354368885
glc__D_e_1:  -2.440825112494011
glc__D_e_1:  -1.7440512797606484
glc__D_e_1:  -3.984218609975056
glc__D_e_1:  -2.3490253250581707
glc__D_e_1:  -2.351657418763623
glc__D_e_1:  -2.116365549644208
glc__D_e_1:  -7.07104231041718
glc__D_e_1:  -1.6488524558539712
glc__D_e_1:  -2.7706390872046365
glc__D_e_1:  -3.762267604994795
glc__D_e_1:  -0.808021557906677
glc__D_e_1:  -3.220737615920772
glc__D_e_1:  -3.303390852845299
glc__D_e_1:  -3.1915277332091536
strain_number:  0 4.066666666666666
strain_various:  0 2.434930982366623
strain_number: 

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8, 13, 17,  1, 16,  3,  6, 20, 19,  2,  6, 11, 11, 12, 19,  4, 18,
       15, 13,  8,  8, 17,  5, 13,  6,  4, 20,  6, 15,  9]), 1: array([11,  9, 12, 13,  2, 16,  9,  9, 10, 13,  8,  1, 20,  7, 18, 10, 12,
        3, 16, 16,  1,  1, 10, 14, 10,  6, 15, 13,  1,  3])}
1
strain_number:  0 12.6
strain_various:  0 6.9214641996232755
strain_number:  1 9.633333333333333
strain_various:  1 5.363353014258483
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8, 13, 17,  1, 16,  3,  6, 20, 19,  2,  6, 11, 11, 12, 19,  4, 18,
       15, 13,  8,  8, 17,  5, 13,  6,  4, 20,  6, 15,  9]), 1: array([11,  9, 12, 13,  2, 16,  9,  9, 10, 13,  8,  1, 20,  7, 18, 10, 12,
        3, 16, 16,  1,  1, 10, 14, 10,  6, 15, 13,  1,  3])}
_2:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 5: [0.64592653

glc__D_e_1:  -8.940248199774672
glc__D_e_1:  -0.40951478013494214
glc__D_e_1:  -1.0470821075085417
glc__D_e_1:  -22.18891681917083
glc__D_e_1:  -28.304027606601707
glc__D_e_1:  -0.5099187212379661
glc__D_e_1:  -30.076271143528444
glc__D_e_1:  -18.492063972251742
glc__D_e_1:  -14.224570081524437
glc__D_e_1:  -18.053276081118007
glc__D_e_1:  -0.3231351934060367
glc__D_e_1:  -4.520155721782327
strain_number:  0 44.56666666666667
strain_various:  0 18.959049437024934
strain_number:  1 0.9666666666666667
strain_various:  1 1.0482790129010926
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 875.0363704278179, 399.128976286871, 420.70587088350254, 416.07284864258554, 415.4904539174276, 447.3987988490005, 417.08793724853484, 428.8907043637669, 469.92113841567306, 489.99009902348683, 446.1593381335818, 386.4254899238851, 374.69376591481966]
slip_r:  374.69376591481966
15
glc__D_e_1:  -24.83843256969789
glc__D_e_1:  -12.010323153178575
glc__D_e_1:  -15.742923942407918
glc__D_e_1:  -29.63981717878705
g

glc__D_e_1:  -0.7891263407243652
glc__D_e_1:  -1.7380567911081948
glc__D_e_1:  -0.14635521565255705
glc__D_e_1:  -0.7277467715497045
glc__D_e_1:  -1.0745950606555266
glc__D_e_1:  -1.8622166535527656
glc__D_e_1:  -1.843711698522669
glc__D_e_1:  -1.0993824019661695
glc__D_e_1:  -2.442144606324555
glc__D_e_1:  -1.4894732550849805
glc__D_e_1:  -0.3292404765958611
glc__D_e_1:  -1.089884416639603
glc__D_e_1:  -0.27762651106616076
glc__D_e_1:  -0.020560639927498903
glc__D_e_1:  -0.5323874080178551
glc__D_e_1:  -1.4330904894214003
glc__D_e_1:  -1.2038221488531942
glc__D_e_1:  -2.746503765888923
glc__D_e_1:  -1.1417905499553032
glc__D_e_1:  -1.9909494189594037
glc__D_e_1:  -4.098711932714151
glc__D_e_1:  -2.404734132106501
glc__D_e_1:  -2.4336818364297104
glc__D_e_1:  -2.5320579798523415
glc__D_e_1:  -2.2069158765081114
strain_number:  0 2.6666666666666665
strain_various:  0 1.6599866130651644
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 875.0363704278

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  1, 20, 18,  7, 10,  9,  5, 10,  8, 19, 12, 13, 10,  7,  6,  4,
       12, 11,  6, 20, 15, 12, 14, 16, 17,  2, 11, 18,  8]), 1: array([ 9,  8, 20,  8,  2, 20,  3, 10, 12, 12, 16,  1, 13, 20,  8, 14, 17,
       19, 10, 15, 10, 11, 20,  9,  6,  4,  5, 17,  1, 17])}
1
strain_number:  0 12.766666666666667
strain_various:  0 6.221914674939075
strain_number:  1 11.233333333333333
strain_various:  1 5.897362875802106
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7,  1, 20, 18,  7, 10,  9,  5, 10,  8, 19, 12, 13, 10,  7,  6,  4,
       12, 11,  6, 20, 15, 12, 14, 16, 17,  2, 11, 18,  8]), 1: array([ 9,  8, 20,  8,  2, 20,  3, 10, 12, 12, 16,  1, 13, 20,  8, 14, 17,
       19, 10, 15, 10, 11, 20,  9,  6,  4,  5, 17,  1, 17])}
_2:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0

glc__D_e_1:  -0.2559941119611753
glc__D_e_1:  -17.1909192845864
glc__D_e_1:  -29.977076868053764
glc__D_e_1:  -4.904991794223193
glc__D_e_1:  -26.116824028994742
glc__D_e_1:  -21.76197834215798
glc__D_e_1:  -0.2832183430550117
glc__D_e_1:  -0.6871307911274007
glc__D_e_1:  -13.974538852300894
glc__D_e_1:  -22.499716805285328
glc__D_e_1:  -0.3522827577498475
glc__D_e_1:  -0.07694195489809919
glc__D_e_1:  -12.322202269942526
glc__D_e_1:  -24.194403002381332
glc__D_e_1:  -17.09342749044941
glc__D_e_1:  -44.26584197869111
glc__D_e_1:  -12.179336811648753
glc__D_e_1:  -36.92188545928184
glc__D_e_1:  -0.10482783257153061
glc__D_e_1:  -18.08190075132582
glc__D_e_1:  -14.408435720153433
glc__D_e_1:  -1.0660548502907092
glc__D_e_1:  -27.38030973807752
strain_number:  0 41.3
strain_various:  0 15.384299355728446
strain_number:  1 1.8
strain_various:  1 1.6411378166788229
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 906.9627167208434, 385.5723493482781, 381.53145210053117, 438.53070248464985, 495.63

glc__D_e_1:  -3.2830789272386256
glc__D_e_1:  -3.8688088143197237
glc__D_e_1:  -4.258658861329904
glc__D_e_1:  -2.1276624398713326
glc__D_e_1:  -2.6866005990754944
glc__D_e_1:  -2.2748028006596446
glc__D_e_1:  -0.4269105215946929
glc__D_e_1:  -1.709425406957875
glc__D_e_1:  -2.128210890800335
glc__D_e_1:  -1.2592088753749069
glc__D_e_1:  -1.8519102937418193
glc__D_e_1:  -1.9002657949673316
glc__D_e_1:  -2.271700157936705
glc__D_e_1:  -2.3923427073473897
glc__D_e_1:  -1.6723618232416873
glc__D_e_1:  -1.6236811506285533
glc__D_e_1:  -0.24516710631101735
strain_number:  0 4.4
strain_various:  0 2.169485346036397
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 906.9627167208434, 385.5723493482781, 381.53145210053117, 438.53070248464985, 495.6316905316905, 426.34928877399295, 433.85900473143937, 469.1282159215983, 476.26684793906895, 400.34096675072857, 379.10719401211077, 333.6757320516927, 382.9914284982529, 333.5284899069895, 381.1865776369681, 233

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20, 16, 14, 17, 14,  7,  2, 18, 18,  3,  7,  5, 10, 17, 14, 11, 14,
        9, 20,  9, 12, 11, 19, 13,  7,  4, 19, 20,  3,  6]), 1: array([17,  8, 17,  6,  5,  6, 15, 15, 11,  3, 11,  2, 13, 12,  7,  9,  2,
        9,  8,  7,  1, 19, 20,  3, 17,  6, 17, 19,  1, 14])}
1
strain_number:  0 13.9
strain_various:  0 6.793869785819173
strain_number:  1 10.0
strain_various:  1 5.853773711604051
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20, 16, 14, 17, 14,  7,  2, 18, 18,  3,  7,  5, 10, 17, 14, 11, 14,
        9, 20,  9, 12, 11, 19, 13,  7,  4, 19, 20,  3,  6]), 1: array([17,  8, 17,  6,  5,  6, 15, 15, 11,  3, 11,  2, 13, 12,  7,  9,  2,
        9,  8,  7,  1, 19, 20,  3, 17,  6, 17, 19,  1, 14])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.28

glc__D_e_1:  -1.1590629935312202
glc__D_e_1:  -27.2926046721774
glc__D_e_1:  -21.837699846055944
glc__D_e_1:  -12.400783965608838
glc__D_e_1:  -8.923010607254664
glc__D_e_1:  -12.146931146577419
glc__D_e_1:  -8.098759263325515
glc__D_e_1:  -26.724234071725533
glc__D_e_1:  -20.82968691129294
glc__D_e_1:  -25.811939710939868
glc__D_e_1:  -35.05088845403945
glc__D_e_1:  -12.078672205145127
glc__D_e_1:  -1.3721379345611866
glc__D_e_1:  -33.20261773796027
glc__D_e_1:  -13.793360371194924
glc__D_e_1:  -17.6403255765474
glc__D_e_1:  -36.45837232750455
glc__D_e_1:  -0.516807825636409
glc__D_e_1:  -0.7872451145936182
glc__D_e_1:  -21.2238486684869
glc__D_e_1:  -12.72746215605438
glc__D_e_1:  -0.4036790599397584
glc__D_e_1:  -1.0079944386619135
glc__D_e_1:  -18.855080663122063
glc__D_e_1:  -23.61762984444259
glc__D_e_1:  -3.3432186808516224
glc__D_e_1:  -11.066877374885769
glc__D_e_1:  -19.508554591554567
glc__D_e_1:  -4.4565278740559915
glc__D_e_1:  -16.85364522607632
strain_number:  0 39.2
str

glc__D_e_1:  -1.9043478627222614
glc__D_e_1:  -2.1616477831113396
glc__D_e_1:  -1.3672913775408102
glc__D_e_1:  -2.125996450580637
glc__D_e_1:  -1.6125181270006466
glc__D_e_1:  -0.8627736861862689
glc__D_e_1:  -2.8649725194712494
glc__D_e_1:  -2.312734575283941
glc__D_e_1:  -1.7581655837921
glc__D_e_1:  -3.3810651574490853
glc__D_e_1:  -1.9817576768036518
strain_number:  0 3.933333333333333
strain_various:  0 2.322833518691246
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 870.539183533185, 344.9224886955151, 392.41265058041364, 418.9955383244857, 423.5976150018171, 416.5271992036948, 386.67835033586186, 405.41914274414205, 420.21692209353927, 403.37705749470064, 374.18854334274727, 416.9126485808336, 305.6711714528872, 345.4438093652957, 219.03521615425348, 230.33752387608055, 206.32493164730008, 209.9561688311688]
slip_r:  209.9561688311688
20
glc__D_e_1:  -0.2378115936606594
glc__D_e_1:  -0.18743690388908307
glc__D_e_1:  -1.7136891798591243
g

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  4,  6,  5,  9, 20, 13, 17, 13, 13,  7, 19, 19, 13,  2, 18,  7,
       20, 17, 10,  1,  2, 16, 12,  1,  3, 10, 16,  4, 13]), 1: array([ 6,  1,  6, 13, 15,  3, 17,  8,  7, 17, 11, 12,  3, 20, 13, 15,  8,
        7,  6, 13, 16,  3, 18,  6,  4,  4,  1,  2, 15, 17])}
1
strain_number:  0 12.1
strain_various:  0 7.46703868120868
strain_number:  1 9.566666666666666
strain_various:  1 5.725285048701845
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  4,  6,  5,  9, 20, 13, 17, 13, 13,  7, 19, 19, 13,  2, 18,  7,
       20, 17, 10,  1,  2, 16, 12,  1,  3, 10, 16,  4, 13]), 1: array([ 6,  1,  6, 13, 15,  3, 17,  8,  7, 17, 11, 12,  3, 20, 13, 15,  8,
        7,  6, 13, 16,  3, 18,  6,  4,  4,  1,  2, 15, 17])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841

glc__D_e_1:  -10.099013418709921
glc__D_e_1:  -0.1423632059360429
glc__D_e_1:  -0.23308517080303115
glc__D_e_1:  -8.240005543417269
glc__D_e_1:  -11.491973941448979
glc__D_e_1:  -0.6860943112209847
glc__D_e_1:  -0.014034054183345246
glc__D_e_1:  -16.82531334687024
glc__D_e_1:  -10.631713121714515
glc__D_e_1:  -11.027154449371157
glc__D_e_1:  -18.742793157362097
glc__D_e_1:  -0.3473961977009683
glc__D_e_1:  -8.256120172241191
glc__D_e_1:  -26.385513027080265
glc__D_e_1:  -0.40059873798600165
glc__D_e_1:  -19.877933339601377
glc__D_e_1:  -15.245742618469801
glc__D_e_1:  -2.0323352818063753
strain_number:  0 39.5
strain_various:  0 18.04208044175246
strain_number:  1 0.3333333333333333
strain_various:  1 0.5374838498865699
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 776.1572274474751, 404.6084130781499, 300.72345417740155, 306.57323378376014, 403.31095937876506, 471.4231389507493, 469.17142477405633, 450.7657044183208, 424.35082448532444, 456.33187694972474, 340.36484434619297, 349.9752947

glc__D_e_1:  -14.859940861873758
glc__D_e_1:  -13.02402979942033
glc__D_e_1:  -7.106884877885283
glc__D_e_1:  -9.21092373210531
glc__D_e_1:  -7.932313863508113
glc__D_e_1:  -9.263222744066688
strain_number:  0 7.766666666666667
strain_various:  0 7.855712712556527
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 776.1572274474751, 404.6084130781499, 300.72345417740155, 306.57323378376014, 403.31095937876506, 471.4231389507493, 469.17142477405633, 450.7657044183208, 424.35082448532444, 456.33187694972474, 340.36484434619297, 349.97529474875057, 322.5126626777161, 255.05010665663087, 265.3809535039709, 142.70631779146396, 201.64739086918104, 193.49825758698432, 204.32815172443657]
slip_r:  204.32815172443657
21
glc__D_e_1:  -14.728816698752928
glc__D_e_1:  -7.142785238053423
glc__D_e_1:  -11.789020992310489
glc__D_e_1:  -1.541320144740288
glc__D_e_1:  -1.712271382012073
glc__D_e_1:  -0.6413054836626506
glc__D_e_1:  -0.28002666428737943
glc__D_e_1:  

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 14,  8,  9, 15,  6, 19, 19, 16, 14, 10,  9,  2, 17, 19, 20,  4,
        4, 19, 15,  8, 15, 16, 14, 20,  1, 17,  5,  2, 19]), 1: array([12, 12, 10, 11,  3,  7, 20, 14, 11,  8,  2, 19,  1,  4, 10, 12, 13,
        2, 15,  4, 14,  2,  6,  8,  7,  2, 14,  6,  3, 12])}
1
strain_number:  0 13.9
strain_various:  0 7.480418883101846
strain_number:  1 8.8
strain_various:  1 5.173006862551025
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 14,  8,  9, 15,  6, 19, 19, 16, 14, 10,  9,  2, 17, 19, 20,  4,
        4, 19, 15,  8, 15, 16, 14, 20,  1, 17,  5,  2, 19]), 1: array([12, 12, 10, 11,  3,  7, 20, 14, 11,  8,  2, 19,  1,  4, 10, 12, 13,
        2, 15,  4, 14,  2,  6,  8,  7,  2, 14,  6,  3, 12])}
_2:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.285

glc__D_e_1:  -14.810364783442989
glc__D_e_1:  -29.640569476641005
glc__D_e_1:  -12.51540040761946
glc__D_e_1:  -31.569378692474665
glc__D_e_1:  -19.025231753754813
glc__D_e_1:  -44.86744419416811
glc__D_e_1:  -19.519964711173483
glc__D_e_1:  -44.21408488782949
glc__D_e_1:  -15.026058888444732
glc__D_e_1:  -31.869649513425728
glc__D_e_1:  -18.19733737349017
glc__D_e_1:  -16.497478513706415
glc__D_e_1:  -43.681018890744845
glc__D_e_1:  -13.238823764821197
glc__D_e_1:  -26.81134491197727
glc__D_e_1:  -27.15423342169727
glc__D_e_1:  -26.642067034414282
glc__D_e_1:  -15.999972913638084
glc__D_e_1:  -21.976501884299772
glc__D_e_1:  -0.7958922481252841
glc__D_e_1:  -12.140401413200529
strain_number:  0 41.36666666666667
strain_various:  0 18.703802346641236
strain_number:  1 0.6666666666666666
strain_various:  1 1.1352924243950933
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 946.0245993042047, 389.65395459803364, 364.4963972431078, 401.7394726689082, 393.7808532218555, 456.7919832784909, 390.28

glc__D_e_1:  -3.2837090539358194
glc__D_e_1:  -6.185559342871582
strain_number:  0 4.2
strain_various:  0 2.0558858593479035
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 946.0245993042047, 389.65395459803364, 364.4963972431078, 401.7394726689082, 393.7808532218555, 456.7919832784909, 390.2804443455373, 456.182101024731, 416.33553985084257, 374.0906175323248, 363.476265247118, 337.26404772214386, 315.2715853014282, 269.04949048133057, 262.6549889935253, 185.9142495158055, 187.55416695877227, 177.81331168831167]
slip_r:  177.81331168831167
20
glc__D_e_1:  -3.1081879387863838
glc__D_e_1:  -2.5637753174230657
glc__D_e_1:  -1.9290293457332777
glc__D_e_1:  -0.35347293908215516
glc__D_e_1:  -0.1565747692251498
glc__D_e_1:  -0.1287955726122847
glc__D_e_1:  -0.3987328519619201
glc__D_e_1:  -0.8444370724355891
glc__D_e_1:  -0.5369631040002008
glc__D_e_1:  -0.848414816742245
glc__D_e_1:  -0.40454248094161804
glc__D_e_1:  -1.1139154448232758
glc__D_e_1:  

In [12]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6,  9, 20, 11, 15,  3, 12, 20,  9, 19,  5,  6, 15,  3,  9, 17,  7,
        1, 16, 20, 15,  5, 16, 16,  8, 14,  6, 15,  2,  4]), 1: array([ 5, 18, 18,  4,  5,  4,  6, 11, 11,  4,  3, 10,  2,  8,  1,  5,  8,
        8, 13, 19,  9,  5, 11,  6,  9,  1,  3,  5,  6, 11])}
1
strain_number:  0 12.633333333333333
strain_various:  0 7.134345722551501
strain_number:  1 7.633333333333334
strain_various:  1 4.736266696695006
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 6,  9, 20, 11, 15,  3, 12, 20,  9, 19,  5,  6, 15,  3,  9, 17,  7,
        1, 16, 20, 15,  5, 16, 16,  8, 14,  6, 15,  2,  4]), 1: array([ 5, 18, 18,  4,  5,  4,  6, 11, 11,  4,  3, 10,  2,  8,  1,  5,  8,
        8, 13, 19,  9,  5, 11,  6,  9,  1,  3,  5,  6, 11])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.285739174469363

glc__D_e_1:  -20.557640174138893
glc__D_e_1:  -8.987671188369958
glc__D_e_1:  -18.068837662193694
glc__D_e_1:  -1.1620252134912867
glc__D_e_1:  -26.047546158039264
glc__D_e_1:  -14.597228505547763
glc__D_e_1:  -28.43295111452557
glc__D_e_1:  -13.257605052351135
glc__D_e_1:  -22.306832993150206
glc__D_e_1:  -35.32877915515181
glc__D_e_1:  -15.361993948668928
glc__D_e_1:  -24.7442703343979
glc__D_e_1:  -33.039072273787454
glc__D_e_1:  -23.88794846806387
glc__D_e_1:  -17.56808432348954
glc__D_e_1:  -26.57757162428789
glc__D_e_1:  -27.720146150987073
glc__D_e_1:  -32.83025701329902
glc__D_e_1:  -19.083324982085166
glc__D_e_1:  -17.216420081867412
glc__D_e_1:  -21.660896784954257
glc__D_e_1:  -15.263077363439521
glc__D_e_1:  -17.817766254387813
glc__D_e_1:  -11.246740255038716
glc__D_e_1:  -17.500866023233808
glc__D_e_1:  -20.142797463033403
strain_number:  0 39.56666666666667
strain_various:  0 19.74619513279007
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
f

glc__D_e_1:  -0.19917619626711003
glc__D_e_1:  -0.15151225590375367
glc__D_e_1:  -1.0580815952418003
glc__D_e_1:  -0.4657386751652294
glc__D_e_1:  -2.04471135241561
glc__D_e_1:  -2.17993907320012
glc__D_e_1:  -1.1444324428695345
glc__D_e_1:  -3.6991719849981837
glc__D_e_1:  -0.6009281767684884
glc__D_e_1:  -0.8184286016819693
glc__D_e_1:  -0.06998539463365838
glc__D_e_1:  -0.6253445475848165
glc__D_e_1:  -1.05089823057476
glc__D_e_1:  -0.8511852385201264
glc__D_e_1:  -10.576042515278514
glc__D_e_1:  -9.65436920729356
glc__D_e_1:  -8.099209506996452
strain_number:  0 2.8333333333333335
strain_various:  0 3.6063216101112716
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 750.1964460977619, 342.608480335454, 462.3334582084583, 467.0202478148149, 390.8626096126097, 388.3872820806644, 443.8236820178883, 462.9107754605995, 415.6581182467066, 331.2311840934458, 317.2554497267537, 321.9943150359727, 271.6331057480776, 232.70256834540152, 196.069673348224

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9, 17, 15, 11,  4, 12,  5, 14,  1, 12, 16,  4, 11,  4,  3, 11, 11,
       11,  3, 13,  4, 12,  7, 20, 17, 10, 14, 15,  4,  3]), 1: array([10, 20,  1,  2, 20,  5, 10, 10, 15,  3,  1, 13,  3,  9, 11, 11,  1,
        2, 18,  6, 16,  9,  4, 12, 20, 20, 17, 16, 17, 19])}
1
strain_number:  0 11.3
strain_various:  0 6.2617888817813085
strain_number:  1 10.7
strain_various:  1 6.553116307020145
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 9, 17, 15, 11,  4, 12,  5, 14,  1, 12, 16,  4, 11,  4,  3, 11, 11,
       11,  3, 13,  4, 12,  7, 20, 17, 10, 14, 15,  4,  3]), 1: array([10, 20,  1,  2, 20,  5, 10, 10, 15,  3,  1, 13,  3,  9, 11, 11,  1,
        2, 18,  6, 16,  9,  4, 12, 20, 20, 17, 16, 17, 19])}
_2:  {0: [0.6345187588838144, 0.2857391744693639], 1: [0.6345187588838144, 0.2857391744693639], 2: [0.6345187588838144, 0.2857391744693639], 3: [0.6345187588838144, 0.2857391744693639], 4: [0.6345187588838144, 0.2857391744693639], 5: [0.6345187588838144, 0.2

glc__D_e_1:  -12.831105467921372
glc__D_e_1:  -15.727744748992924
glc__D_e_1:  -13.861843804498571
glc__D_e_1:  -19.61871201482086
glc__D_e_1:  -21.726370396895472
glc__D_e_1:  -23.66953578355486
glc__D_e_1:  -3.16139271947862
glc__D_e_1:  -13.270440093396289
glc__D_e_1:  -13.36817509296065
glc__D_e_1:  -23.310362947009857
glc__D_e_1:  -16.434357613978758
glc__D_e_1:  -0.21152917449223807
glc__D_e_1:  -11.981612653935569
glc__D_e_1:  -17.466376878026608
glc__D_e_1:  -5.245383943227289
glc__D_e_1:  -14.405829423367543
glc__D_e_1:  -0.3872661675789004
glc__D_e_1:  -3.742620431751709
glc__D_e_1:  -11.965260465398858
glc__D_e_1:  -22.342700961516552
glc__D_e_1:  -4.6582263199961425
glc__D_e_1:  -3.899719604145214
glc__D_e_1:  -9.979995096730896
strain_number:  0 40.3
strain_various:  0 21.084196293274573
strain_number:  1 0.5666666666666667
strain_various:  1 1.4302291968616625
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 716.8650934333983, 356.5474995850383, 391.2054929719404, 446.590275436

glc__D_e_1:  -0.3510917329833517
glc__D_e_1:  -1.4071957313393686
glc__D_e_1:  -0.8764867662424796
glc__D_e_1:  -1.62247573702493
glc__D_e_1:  -1.053820731750807
glc__D_e_1:  -3.209624601251737
glc__D_e_1:  -0.84936406362739
glc__D_e_1:  -3.405193503200767
glc__D_e_1:  -5.429956099249134
glc__D_e_1:  -5.295148691788508
glc__D_e_1:  -7.651444231240521
glc__D_e_1:  -3.30184863265251
glc__D_e_1:  -7.502591232695587
glc__D_e_1:  -6.126371084073465
glc__D_e_1:  -5.62512838372508
glc__D_e_1:  -2.008593813944913
glc__D_e_1:  -1.5014708309776978
glc__D_e_1:  -0.7777037730490678
glc__D_e_1:  -0.5971654824410924
glc__D_e_1:  -0.935641340898484
strain_number:  0 3.3
strain_various:  0 3.0127506811328857
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 716.8650934333983, 356.5474995850383, 391.2054929719404, 446.59027543663774, 444.79749818894567, 416.03948686792495, 439.3283642456393, 412.0507776136983, 424.1336887039732, 406.5405719297317, 367.3103837484931

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  1,  9,  8, 19,  8, 12, 20,  2,  3, 13, 13, 15,  9,  2,  4, 18,
       16,  2, 15, 18, 13, 18,  1, 16,  5,  4,  9, 10, 12]), 1: array([ 3, 15, 20, 11,  3, 14, 12,  7,  6,  1, 18, 20, 11, 20, 14, 20, 14,
        5,  2,  1, 20,  9,  7, 19,  9,  6,  9, 11,  6, 11])}
1
strain_number:  0 11.433333333333334
strain_various:  0 7.301521911370412
strain_number:  1 10.8
strain_various:  1 6.123180001709351
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  1,  9,  8, 19,  8, 12, 20,  2,  3, 13, 13, 15,  9,  2,  4, 18,
       16,  2, 15, 18, 13, 18,  1, 16,  5,  4,  9, 10, 12]), 1: array([ 3, 15, 20, 11,  3, 14, 12,  7,  6,  1, 18, 20, 11, 20, 14, 20, 14,
        5,  2,  1, 20,  9,  7, 19,  9,  6,  9, 11,  6, 11])}
_2:  {0: [0.6345187588838144, 0.2857391744693639], 1: [0.6345187588838144, 0.2857391744693639], 2: [0.6345187588838144, 0.2857391744693639], 3: [0.6345187588838144, 0.2857391744693639], 4: [0.6345187588838144, 0.2857391744693639], 5: [0.63451875

glc__D_e_1:  -15.450556298040302
glc__D_e_1:  -17.74870609046593
glc__D_e_1:  -0.24162797434564998
glc__D_e_1:  -10.244450479088254
glc__D_e_1:  -35.964209609904415
glc__D_e_1:  -1.7847185572656628
glc__D_e_1:  -1.0053648208386277
glc__D_e_1:  -9.937005055050498
glc__D_e_1:  -24.25777391730972
glc__D_e_1:  -1.6943378983613897
glc__D_e_1:  -1.0498459294796691
glc__D_e_1:  -6.767939352099343
glc__D_e_1:  -33.57577747909774
glc__D_e_1:  -3.13614274542223
glc__D_e_1:  -0.6271451674391728
glc__D_e_1:  -10.629967672181778
glc__D_e_1:  -22.792281394145906
glc__D_e_1:  -2.4331105588954536
glc__D_e_1:  -12.847309089822597
glc__D_e_1:  -28.230117797804198
glc__D_e_1:  -0.3912343246272556
glc__D_e_1:  -0.023555559873927545
glc__D_e_1:  -20.202609634658494
glc__D_e_1:  -20.505725733368553
glc__D_e_1:  -17.49395256665206
glc__D_e_1:  -8.77503001095305
glc__D_e_1:  -28.615737554422434
glc__D_e_1:  -10.029216051972105
glc__D_e_1:  -20.15984536167853
glc__D_e_1:  -30.218479774882226
glc__D_e_1:  -17.1

glc__D_e_1:  -0.981686620029558
glc__D_e_1:  -1.7631803580514296
glc__D_e_1:  -3.703675191182758
glc__D_e_1:  -1.8553178939073665
glc__D_e_1:  -7.09516763607609
glc__D_e_1:  -2.029574226781978
strain_number:  0 4.2
strain_various:  0 5.36283507111677
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 938.1028423331056, 348.03858750898235, 343.3991633366633, 393.9873413866835, 456.9464130238749, 455.76085865888507, 438.1901095403855, 370.3433242358527, 371.8362430723499, 410.5124191420693, 435.30383243693274, 309.64032611618495, 354.5452941474422, 431.31884787829625, 287.4685560080459, 249.73337739095035, 246.60058655883375, 180.01744944553164, 216.88415750915752]
slip_r:  216.88415750915752
21
glc__D_e_1:  -12.852765313405476
glc__D_e_1:  -4.186305082912855
glc__D_e_1:  -5.603314968237688
glc__D_e_1:  -0.537420057282181
glc__D_e_1:  -1.7833332139849993
glc__D_e_1:  -0.7229222569517408
glc__D_e_1:  -0.2000666876607844
glc__D_e_1:  -0.3060508597001059

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 18, 16, 20, 11, 10,  2, 10,  8, 12, 18, 13, 16,  7,  4, 18, 14,
        6, 18,  1, 11,  7, 14, 20,  5, 13, 18, 10,  3, 12]), 1: array([18, 14,  4, 11, 12,  2, 12,  2,  9, 18, 13,  7,  4, 11,  6,  6, 16,
        8, 18,  3,  8, 17,  7,  1, 10,  3, 12, 16, 19,  6])}
1
strain_number:  0 13.1
strain_various:  0 6.7889616289974715
strain_number:  1 9.766666666666667
strain_various:  1 5.432515275838829
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 18, 16, 20, 11, 10,  2, 10,  8, 12, 18, 13, 16,  7,  4, 18, 14,
        6, 18,  1, 11,  7, 14, 20,  5, 13, 18, 10,  3, 12]), 1: array([18, 14,  4, 11, 12,  2, 12,  2,  9, 18, 13,  7,  4, 11,  6,  6, 16,
        8, 18,  3,  8, 17,  7,  1, 10,  3, 12, 16, 19,  6])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -7.2061182115956175
glc__D_e_1:  -23.86476688450674
glc__D_e_1:  -1.5452416357143655
glc__D_e_1:  -10.386885621930494
glc__D_e_1:  -15.595504843525863
glc__D_e_1:  -2.215061252251605
glc__D_e_1:  -12.1808363614124
strain_number:  0 48.0
strain_various:  0 15.438048235879215
strain_number:  1 0.8666666666666667
strain_various:  1 0.9213516640723501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 758.4176476258518, 368.7804820953041, 391.46265795607906, 431.7082018858335, 432.8500043815834, 450.9922293858478, 454.1084667787618, 423.4584352885911, 471.8583380989133, 391.45233353560684, 390.84708834083125, 398.42082333431387, 375.84989606648253]
slip_r:  375.84989606648253
15
glc__D_e_1:  -11.379159966028656
glc__D_e_1:  -12.460079016434571
glc__D_e_1:  -13.021588711245913
glc__D_e_1:  -9.999842787376704
glc__D_e_1:  -29.131751799414047
glc__D_e_1:  -21.467835025515186
glc__D_e_1:  -23.85884990021464
glc__D_e_1:  -18.9449622954694
glc__D_e_1:  -28.502156145828955
glc__D_e_1:  -39.0

glc__D_e_1:  -0.12632012620160094
glc__D_e_1:  -0.8461543587572948
glc__D_e_1:  -0.4785378822412161
glc__D_e_1:  -0.9499144590541004
glc__D_e_1:  -0.7538851033689076
glc__D_e_1:  -2.1902315805526698
glc__D_e_1:  -2.2502854090632463
glc__D_e_1:  -2.0566081690297335
glc__D_e_1:  -1.1158677688701055
glc__D_e_1:  -1.9809484079439845
glc__D_e_1:  -0.9766102464595473
glc__D_e_1:  -1.7286014057356416
glc__D_e_1:  -0.24813262703159733
glc__D_e_1:  -2.2439896870760556
glc__D_e_1:  -0.43937526536698723
glc__D_e_1:  -3.178416569231308
glc__D_e_1:  -2.2781357736486614
glc__D_e_1:  -2.0522536258012556
glc__D_e_1:  -1.48257275332512
glc__D_e_1:  -2.302833833471917
glc__D_e_1:  -1.0890016884725415
glc__D_e_1:  -0.13513653187068386
glc__D_e_1:  -1.4378083331166265
glc__D_e_1:  -1.4467798052962464
glc__D_e_1:  -2.0510883876181865
glc__D_e_1:  -0.7223345274222036
glc__D_e_1:  -1.1351427339013
strain_number:  0 2.8
strain_various:  0 1.2754084313139327
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_th

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6, 19,  4, 19, 16,  1, 18, 20, 18, 13,  5,  3, 12,  5, 18,  8,  4,
       13, 17, 19, 10, 11,  6,  9, 17, 13,  6, 20,  6,  6]), 1: array([ 5,  3,  3, 15,  3,  9, 17, 20, 20,  5, 11,  6, 14, 14, 15,  7, 11,
       18, 18,  9, 10, 10, 14,  7,  8, 15, 18, 14,  7,  4])}
1
strain_number:  0 13.266666666666667
strain_various:  0 7.145783527523222
strain_number:  1 11.0
strain_various:  1 5.272570530585627
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 6, 19,  4, 19, 16,  1, 18, 20, 18, 13,  5,  3, 12,  5, 18,  8,  4,
       13, 17, 19, 10, 11,  6,  9, 17, 13,  6, 20,  6,  6]), 1: array([ 5,  3,  3, 15,  3,  9, 17, 20, 20,  5, 11,  6, 14, 14, 15,  7, 11,
       18, 18,  9, 10, 10, 14,  7,  8, 15, 18, 14,  7,  4])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -1.6331273496597767
glc__D_e_1:  -31.444621938216407
glc__D_e_1:  -25.017823003702603
glc__D_e_1:  -0.509276994347097
glc__D_e_1:  -29.509538653682043
glc__D_e_1:  -20.92862826007497
glc__D_e_1:  -0.31244833867881283
glc__D_e_1:  -0.5900267066407107
glc__D_e_1:  -21.139019587757634
glc__D_e_1:  -17.186663112718968
glc__D_e_1:  -26.14156038335386
glc__D_e_1:  -30.716064918041692
glc__D_e_1:  -1.4634801570287337
glc__D_e_1:  -20.064137237778095
glc__D_e_1:  -20.093066823135636
glc__D_e_1:  -0.8142349986045012
glc__D_e_1:  -11.652279630718889
glc__D_e_1:  -20.09254799692414
glc__D_e_1:  -0.8137161723930042
glc__D_e_1:  -10.867687054107403
strain_number:  0 37.666666666666664
strain_various:  0 13.125886213467222
strain_number:  1 1.7333333333333334
strain_various:  1 1.6110727964792761
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 669.5343491647671, 349.1344335489073, 363.21372530977794, 417.6138387197599, 467.09483177641073, 462.927828329802, 455.8131874276062, 437.568723654395

glc__D_e_1:  -1.6995771011399143
glc__D_e_1:  -1.2724925170935335
glc__D_e_1:  -0.28372256365428683
glc__D_e_1:  -0.9524035964597997
glc__D_e_1:  -1.8868342211546132
glc__D_e_1:  -3.407601072833593
glc__D_e_1:  -2.8288561615200885
glc__D_e_1:  -4.466729375128894
glc__D_e_1:  -2.0955960352925795
glc__D_e_1:  -0.8857222088638086
glc__D_e_1:  -2.029873698987833
glc__D_e_1:  -0.811496430677166
glc__D_e_1:  -3.2883082426835437
glc__D_e_1:  -3.079514363683778
glc__D_e_1:  -5.093200318401559
glc__D_e_1:  -1.5067186434212458
glc__D_e_1:  -4.67441202623111
glc__D_e_1:  -5.689562325785803
glc__D_e_1:  -2.395902598413622
strain_number:  0 4.033333333333333
strain_various:  0 2.1982316125063397
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 669.5343491647671, 349.1344335489073, 363.21372530977794, 417.6138387197599, 467.09483177641073, 462.927828329802, 455.8131874276062, 437.56872365439546, 427.0696550248141, 397.05709954927374, 410.4526900334215, 326.2205

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([20, 13,  4,  7, 15, 16,  9, 14, 10,  9, 14, 19,  2, 14,  6, 19,  5,
       16,  1, 14, 14,  4, 16,  5, 18,  8,  4, 19, 18, 13]), 1: array([ 1,  7, 12, 12,  7, 19, 19, 10, 15,  1,  6,  7,  3, 20,  7,  5,  7,
       14,  5,  7, 19, 10, 19, 10, 17, 20, 12, 16,  1, 14])}
1
strain_number:  0 13.333333333333334
strain_various:  0 6.774133810967193
strain_number:  1 10.733333333333333
strain_various:  1 5.982938705649219
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([20, 13,  4,  7, 15, 16,  9, 14, 10,  9, 14, 19,  2, 14,  6, 19,  5,
       16,  1, 14, 14,  4, 16,  5, 18,  8,  4, 19, 18, 13]), 1: array([ 1,  7, 12, 12,  7, 19, 19, 10, 15,  1,  6,  7,  3, 20,  7,  5,  7,
       14,  5,  7, 19, 10, 19, 10, 17, 20, 12, 16,  1, 14])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -25.396995478097576
glc__D_e_1:  -19.59889540707095
glc__D_e_1:  -37.3389763761489
glc__D_e_1:  -15.3365507471469
glc__D_e_1:  -29.387015128635554
glc__D_e_1:  -34.48713269482729
glc__D_e_1:  -21.70277236469412
glc__D_e_1:  -0.23491977959985455
glc__D_e_1:  -33.49216714481652
glc__D_e_1:  -19.214570936124097
glc__D_e_1:  -0.11873772685855855
glc__D_e_1:  -30.69033233486768
glc__D_e_1:  -29.00047675494803
glc__D_e_1:  -15.248347277770868
glc__D_e_1:  -38.78202636574048
glc__D_e_1:  -1.2590339956419108
glc__D_e_1:  -18.59123660032096
glc__D_e_1:  -36.65361473337401
glc__D_e_1:  -0.6509690754060671
glc__D_e_1:  -17.27095317636507
glc__D_e_1:  -34.810732791597985
glc__D_e_1:  -1.8487805578912777
glc__D_e_1:  -0.9239787370916337
glc__D_e_1:  -14.18811764837989
glc__D_e_1:  -30.80570100639399
glc__D_e_1:  -2.221790293813541
glc__D_e_1:  -37.72741229784341
glc__D_e_1:  -23.661317658164172
glc__D_e_1:  -1.3417924093717986
glc__D_e_1:  -1.3511370113494632
glc__D_e_1:  -13.574582498

glc__D_e_1:  -1.6416192243936139
glc__D_e_1:  -2.451675486929985
glc__D_e_1:  -1.8616033373992047
glc__D_e_1:  -1.7845873120690867
glc__D_e_1:  -2.184235108825109
glc__D_e_1:  -3.2960347576289823
glc__D_e_1:  -2.081051820991938
glc__D_e_1:  -2.353427153196631
glc__D_e_1:  -2.4336456801203505
glc__D_e_1:  -2.406440532795491
glc__D_e_1:  -1.9650593539426564
glc__D_e_1:  -1.6200379374301745
glc__D_e_1:  -1.6778005862153555
glc__D_e_1:  -3.6875949042713567
glc__D_e_1:  -2.5402424698261843
glc__D_e_1:  -1.586812399512638
glc__D_e_1:  -3.9583272876243902
glc__D_e_1:  -1.620252467027877
glc__D_e_1:  -1.1573520824892376
glc__D_e_1:  -2.182957442251893
glc__D_e_1:  -1.0618250977631896
glc__D_e_1:  -1.6663992922182438
glc__D_e_1:  -0.5981339911395334
glc__D_e_1:  -1.9771853706321951
glc__D_e_1:  -1.8878417631136064
glc__D_e_1:  -0.04087467941388079
glc__D_e_1:  -2.5819662221118946
glc__D_e_1:  -0.5799966089084714
glc__D_e_1:  -0.8794958335821753
glc__D_e_1:  -0.9189260758503394
strain_number:  0

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  8, 19, 15,  9,  7, 15,  9, 17,  3, 11,  6, 14,  4,  7,  9,  8,
        2, 19, 11,  6, 16, 11,  3,  8,  4, 18,  7, 11,  5]), 1: array([ 9,  9,  9,  5, 20,  6,  1, 18,  6,  3, 19, 12, 16, 13, 20,  5,  6,
       17, 15, 13,  2,  9, 12, 17,  5,  6, 16, 19, 17,  5])}
1
strain_number:  0 11.1
strain_various:  0 5.889821729050889
strain_number:  1 11.0
strain_various:  1 5.853773711604051
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7,  8, 19, 15,  9,  7, 15,  9, 17,  3, 11,  6, 14,  4,  7,  9,  8,
        2, 19, 11,  6, 16, 11,  3,  8,  4, 18,  7, 11,  5]), 1: array([ 9,  9,  9,  5, 20,  6,  1, 18,  6,  3, 19, 12, 16, 13, 20,  5,  6,
       17, 15, 13,  2,  9, 12, 17,  5,  6, 16, 19, 17,  5])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404

glc__D_e_1:  -30.518720615868475
glc__D_e_1:  -16.734542696278538
glc__D_e_1:  -22.69994523149946
glc__D_e_1:  -1.9007666948377047
glc__D_e_1:  -18.6097362079866
glc__D_e_1:  -14.391953050306691
glc__D_e_1:  -0.525834025865521
glc__D_e_1:  -13.460273166669737
glc__D_e_1:  -5.597008381974746
glc__D_e_1:  -1.5216436787498013
glc__D_e_1:  -5.4011926638677465
glc__D_e_1:  -1.325827960642802
glc__D_e_1:  -4.796991141865533
glc__D_e_1:  -0.9066479068929922
glc__D_e_1:  -0.31708529350040493
glc__D_e_1:  -3.2731566747837597
glc__D_e_1:  -15.739744524837548
glc__D_e_1:  -1.0219528366982686
glc__D_e_1:  -2.321366185734764
glc__D_e_1:  -1.394990788423435
glc__D_e_1:  -1.2119921731578371
glc__D_e_1:  -8.343591801797242
glc__D_e_1:  -3.031794901851036
glc__D_e_1:  -1.0761673403027139
glc__D_e_1:  -1.5618427734696267
strain_number:  0 42.7
strain_various:  0 14.24347803967369
strain_number:  1 1.6333333333333333
strain_various:  1 1.4255603186895398
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 660.853

glc__D_e_1:  -6.795385149254971
glc__D_e_1:  -2.7029768762551076
glc__D_e_1:  -2.7930092247945058
glc__D_e_1:  -3.4879217132571894
glc__D_e_1:  -2.967129256110252
glc__D_e_1:  -2.2464976487717543
glc__D_e_1:  -4.644862161770141
glc__D_e_1:  -2.7764127264001415
glc__D_e_1:  -3.14844027978493
glc__D_e_1:  -1.9135481769921525
glc__D_e_1:  -2.9535462836534196
glc__D_e_1:  -4.703272738820553
glc__D_e_1:  -0.5735526473087289
glc__D_e_1:  -5.2652885717309665
glc__D_e_1:  -1.1134385876858102
glc__D_e_1:  -2.674383855804826
glc__D_e_1:  -4.682532742732264
glc__D_e_1:  -2.4706067198885764
strain_number:  0 4.466666666666667
strain_various:  0 2.0933757957476775
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 660.8530012943711, 327.8197361848678, 299.1173136074453, 400.9176365301365, 465.3910556548714, 440.27299323483527, 445.20559965019174, 418.4421090553094, 431.99454308473526, 387.43433343336045, 395.2515561447218, 329.39542314372875, 374.7393720584558, 

In [12]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7, 19, 10,  4, 10,  8,  2, 15,  4,  7,  9, 13, 20, 18, 18, 15, 20,
       20,  3, 18,  9, 20, 12,  2, 16, 14,  9, 20,  3, 10]), 1: array([19,  1,  3,  9, 19,  4, 13, 12, 19, 15, 13, 10,  8, 19, 15, 17, 16,
        7,  8,  4,  9, 14,  9,  5,  8, 14, 10,  2,  4, 15])}
1
strain_number:  0 13.8
strain_various:  0 7.435948717771884
strain_number:  1 10.7
strain_various:  1 5.3984565695514615
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7, 19, 10,  4, 10,  8,  2, 15,  4,  7,  9, 13, 20, 18, 18, 15, 20,
       20,  3, 18,  9, 20, 12,  2, 16, 14,  9, 20,  3, 10]), 1: array([19,  1,  3,  9, 19,  4, 13, 12, 19, 15, 13, 10,  8, 19, 15, 17, 16,
        7,  8,  4,  9, 14,  9,  5,  8, 14, 10,  2,  4, 15])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988575291567940

glc__D_e_1:  -4.92132210972753
glc__D_e_1:  -7.872582794681776
glc__D_e_1:  -2.093872764060615
glc__D_e_1:  -21.469310136813615
glc__D_e_1:  -23.27456924821586
glc__D_e_1:  -0.2863699509909772
glc__D_e_1:  -8.200367028207884
glc__D_e_1:  -15.737569878873177
glc__D_e_1:  -3.9182743063067083
glc__D_e_1:  -7.7945310820982066
glc__D_e_1:  -3.5424028376995165
glc__D_e_1:  -15.24752425190346
glc__D_e_1:  -6.498119488069267
glc__D_e_1:  -23.279179432079548
glc__D_e_1:  -12.377394882960845
glc__D_e_1:  -23.305097609907396
glc__D_e_1:  -12.599247166397078
glc__D_e_1:  -40.781892204322176
glc__D_e_1:  -17.372384365637597
glc__D_e_1:  -28.180793331291916
glc__D_e_1:  -32.17875444744681
glc__D_e_1:  -28.202337082936445
glc__D_e_1:  -23.54242917489984
glc__D_e_1:  -12.738924826998655
glc__D_e_1:  -26.739117914297793
strain_number:  0 37.53333333333333
strain_various:  0 19.855701672035888
strain_number:  1 1.0666666666666667
strain_various:  1 1.4817407180595248
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1

glc__D_e_1:  -4.4387221736141935
glc__D_e_1:  -4.281290081551443
glc__D_e_1:  -4.92449768044238
glc__D_e_1:  -5.06410381330516
glc__D_e_1:  -3.734804218137701
glc__D_e_1:  -1.9083239413176247
glc__D_e_1:  -7.330065117093245
glc__D_e_1:  -1.087026826394819
glc__D_e_1:  -3.569836861176192
glc__D_e_1:  -0.9878796304971097
glc__D_e_1:  -0.7051999261451978
glc__D_e_1:  -0.2099444129965049
glc__D_e_1:  -0.06684820191797713
glc__D_e_1:  -0.537002666086769
glc__D_e_1:  -0.39698778366299003
glc__D_e_1:  -2.6769663936353605
glc__D_e_1:  -1.3534224619578845
glc__D_e_1:  -1.8970634932175325
glc__D_e_1:  -0.8161867440176871
glc__D_e_1:  -2.5650888023067298
glc__D_e_1:  -1.6846884672921763
strain_number:  0 2.7666666666666666
strain_various:  0 2.836468853267308
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 666.2342385426751, 339.84632898061693, 416.49864353482775, 470.88399655899644, 476.3543112914094, 496.47058257278314, 443.12349883663563, 473.05671605954

In [13]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([15, 13, 20, 16,  4,  1, 11, 17, 19,  3,  6,  6,  2,  8,  4,  7, 14,
        8,  1, 10,  7,  5, 17,  1,  7,  7, 14, 13,  5, 14]), 1: array([19,  6, 18,  3, 13, 14,  5, 10,  5, 11,  5,  1,  3,  5, 15,  1,  4,
        9, 19,  3, 15, 14, 11,  3,  6,  1,  1, 14, 11,  8])}
1
strain_number:  0 10.6
strain_various:  0 6.686304410260325
strain_number:  1 8.433333333333334
strain_various:  1 5.643186649009189
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([15, 13, 20, 16,  4,  1, 11, 17, 19,  3,  6,  6,  2,  8,  4,  7, 14,
        8,  1, 10,  7,  5, 17,  1,  7,  7, 14, 13,  5, 14]), 1: array([19,  6, 18,  3, 13, 14,  5, 10,  5, 11,  5,  1,  3,  5, 15,  1,  4,
        9, 19,  3, 15, 14, 11,  3,  6,  1,  1, 14, 11,  8])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -13.798123584449181
glc__D_e_1:  -10.154313354923055
glc__D_e_1:  -19.562997872921237
glc__D_e_1:  -24.61319059600747
glc__D_e_1:  -6.88981562527265
glc__D_e_1:  -0.08572969557128118
glc__D_e_1:  -9.441696942314604
glc__D_e_1:  -2.5670708724865605
strain_number:  0 39.53333333333333
strain_various:  0 19.172433914926458
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 841.2792667858458, 400.3500216888375, 426.4163029952504, 500.00906593406586, 501.088789353921, 413.89344506472725, 416.8280170082802, 457.2358030858031, 467.02898510999535, 393.69006765228164, 322.23999496039846, 338.06610111405996, 391.1888819215312, 297.22243674290206]
slip_r:  297.22243674290206
16
glc__D_e_1:  -4.193831491855802
glc__D_e_1:  -2.974576728576326
glc__D_e_1:  -6.005342305089151
glc__D_e_1:  -11.09340027755194
glc__D_e_1:  -6.2722589414509216
glc__D_e_1:  -9.155918546528287
glc__D_e_1:  -24.274573162942097
glc__D_e_1:  -18.35573529519728
glc__D_e_1:  -2.

glc__D_e_1:  -17.267569334491373
glc__D_e_1:  -17.015256817799077
glc__D_e_1:  -12.9354816231235
glc__D_e_1:  -5.285463661100941
glc__D_e_1:  -6.451576141688136
glc__D_e_1:  -2.76572499179287
glc__D_e_1:  -3.5440450844054343
glc__D_e_1:  -0.4008633373601779
glc__D_e_1:  -0.811897242544976
glc__D_e_1:  -1.721426690846298
glc__D_e_1:  -0.32332602227852814
glc__D_e_1:  -0.6011734280767137
strain_number:  0 8.833333333333334
strain_various:  0 12.646694781202275
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 841.2792667858458, 400.3500216888375, 426.4163029952504, 500.00906593406586, 501.088789353921, 413.89344506472725, 416.8280170082802, 457.2358030858031, 467.02898510999535, 393.69006765228164, 322.23999496039846, 338.06610111405996, 391.1888819215312, 297.22243674290206, 210.08951925517525, 165.0031251914078, 164.8560265608817, 210.10434638208636, 171.54075189233617, 188.5387302752242, 168.538339248743]
slip_r:  168.538339248743
23
glc__D_e_1:  

strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 841.2792667858458, 400.3500216888375, 426.4163029952504, 500.00906593406586, 501.088789353921, 413.89344506472725, 416.8280170082802, 457.2358030858031, 467.02898510999535, 393.69006765228164, 322.23999496039846, 338.06610111405996, 391.1888819215312, 297.22243674290206, 210.08951925517525, 165.0031251914078, 164.8560265608817, 210.10434638208636, 171.54075189233617, 188.5387302752242, 168.538339248743, 178.87649583444778, 187.04655187433826, 128.74107142857144, 130.21428571428572, 106.0, 30.0, 60.0, 15.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is:  {}
The change process of microorganism distribution is:  {1: {0: [8.979435524822293, 7.482862937351911, 11.972580699763057, 9.478293053979087, 1.9954301166271762, 0.49885752915679404, 6.485147879038323, 9.97715058313588, 10.974865641449469, 1.4965725874703821, 3.4920027040975583, 3.4920027